<a href="https://colab.research.google.com/github/Sanika18/Netflix_data_analysis/blob/main/Netflix_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#for linear algebra
import numpy as np 

#for data preparation
import pandas as pd

# for data visualisation
import plotly.express as px

#for sentiment analysis
from textblob import TextBlob

In [2]:
#importing the data as netflixtitles
netflixtitles = pd.read_csv('/content/netflix_titles.csv')

#displays the number of rows and columns
netflixtitles.shape

(8807, 12)

In [3]:
#displays the names of columns within netflixtitles
netflixtitles.columns

,rating,counts
0,66 min,1
1,74 min,1
2,84 min,1
3,G,41
4,NC-17,3
5,NR,80
6,PG,287
7,PG-13,490
8,R,799
9,TV-14,2160


In [6]:
#dropping netflix ratings called 66min, 74 min, 84 min
netflix2 = netflixtitles.set_index("rating")
netflix_final = netflix2.drop(["66 min", "74 min", "84 min"])

In [9]:
#showing districution of content rating on netflix
z = netflix_final.groupby(['rating']).size().reset_index(name='counts')
pieChart = px.pie(z, values='counts', names='rating', 
                  title='Distribution of Content Ratings on Netflix',
                  color_discrete_sequence=px.colors.qualitative.Set1)
pieChart.show()

In [10]:
netflix_final['director']=netflix_final['director'].fillna('No Director Specified')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
filtered_directors=pd.DataFrame()
filtered_directors=netflix_final['director'].str.split(',',expand=True).stack()
filtered_directors=filtered_directors.to_frame()
filtered_directors.columns=['Director']


In [17]:
directors=filtered_directors.groupby(['Director']).size().reset_index(name='Total Content')
directors=directors[directors.Director !='No Director Specified']
directors=directors.sort_values(by=['Total Content'],ascending=False)
directorsTop5=directors.head()
directorsTop5=directorsTop5.sort_values(by=['Total Content'])
top5directors=px.bar(directorsTop5,x='Total Content',y='Director',title='Top 5 Directors on Netflix based on content', color='Total Content')
top5directors.show()

In [20]:
netflix_final['cast']=netflix_final['cast'].fillna('No Cast Specified')
filtered_actors=pd.DataFrame()
filtered_actors=netflix_final['cast'].str.split(',',expand=True).stack()
filtered_actors=filtered_actors.to_frame()
filtered_actors.columns=['Actor']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
actors=filtered_actors.groupby(['Actor']).size().reset_index(name='Total Content')
actors=actors[actors.Actor !='No Cast Specified']
actors=actors.sort_values(by=['Total Content'],ascending=False)
actorsTop5=actors.head()
actorsTop5=actorsTop5.sort_values(by=['Total Content'])
fig2=px.bar(actorsTop5,x='Total Content',y='Actor', title='Top 5 Actors on Netflix', color='Total Content')
fig2.show()

In [24]:
netflix1=netflix_final[['type','release_year']]
netflix1=netflix1.rename(columns={"release_year": "Release Year"})
netflix2=netflix1.groupby(['Release Year','type']).size().reset_index(name='Total Content')
netflix2=netflix2[netflix2['Release Year']>=2010]
fig3 = px.line(netflix2, x="Release Year", y="Total Content", color='type',title='Trend of content produced over the years on Netflix')
fig3.show()

In [27]:
netflix_sent=netflixtitles[['release_year','description']]
netflix_sent=netflix_sent.rename(columns={'release_year':'Release Year'})
for index,row in netflix_sent.iterrows():
    z=row['description']
    testimonial=TextBlob(z)
    p=testimonial.sentiment.polarity
    if p==0:
        sent='Neutral'
    elif p>0:
        sent='Positive'
    else:
        sent='Negative'
    netflix_sent.loc[[index,2],'Sentiment'] = sent


netflix_sent=netflix_sent.groupby(['Release Year','Sentiment']).size().reset_index(name='Total Content')

netflix_sent=netflix_sent[netflix_sent['Release Year']>=2010]
fig4 = px.bar(netflix_sent, x="Release Year", y="Total Content", color="Sentiment", title="Sentiment of content on Netflix")
fig4.show()